# Set up 

In [1]:
# !cp -r "/content/drive/MyDrive/AffectNet data/AffectNet.h5" /content/AffectNet.h5
# # !cp -r "/content/drive/MyDrive/AffectNet data/balance_affectNet.h5" /content/AffectNet.h5

In [2]:
import os
import numpy as np
import h5py
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *

In [3]:
hf = h5py.File('AffectNet.h5', 'r')
images = hf['images'][:]
hf.close()

print(images.shape)

(414798, 112, 112, 3)


# Define architecture of Deep Convolutional GAN

In [4]:
class DCGAN:

    @staticmethod
    def build_generator(dim, depth, channels=3, inputDim=100, outputDim=512):

        model = Sequential()
        inputShape = (dim, dim, depth)
        chanDim = -1

        model.add(Dense(input_dim=inputDim, units=outputDim))
        model.add(Activation("relu"))
        model.add(BatchNormalization())

        model.add(Dense(dim * dim * depth))
        model.add(Activation("relu"))
        model.add(BatchNormalization())

        model.add(Reshape(inputShape))  # (4, 4, 1024)

        model.add(Conv2DTranspose(512, (4, 4), strides=(2, 2), padding="same")) # (8, 8, 512)
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))

        model.add(Conv2DTranspose(256, (4, 4), strides=(2, 2), padding="same")) # (16, 16, 256)
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))

        model.add(Conv2DTranspose(125, (4, 4), strides=(2, 2), padding="same")) # (32, 32, 128)
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))

        model.add(Conv2DTranspose(channels, (4, 4), strides=(2, 2), padding="same"))
        model.add(Activation("tanh"))
        
        return model

    @staticmethod
    def build_discriminator(width, height, depth, alpha=0.2):

        model = Sequential()

        inputShape = (height, width, depth)

        model.add(Conv2D(128, (4, 4), padding="same", strides=(2, 2), input_shape=inputShape))
        model.add(LeakyReLU(alpha=alpha))

        model.add(Conv2D(256, (4, 4), padding="same", strides=(2, 2)))
        model.add(LeakyReLU(alpha=alpha))

        model.add(Conv2D(512, (4, 4), padding="same", strides=(2, 2)))
        model.add(LeakyReLU(alpha=alpha))

        model.add(Conv2D(1024, (4, 4), padding="same", strides=(2, 2)))
        model.add(LeakyReLU(alpha=alpha))

        model.add(Flatten())
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=alpha))

        model.add(Dense(1))
        model.add(Activation("sigmoid"))

        return model

In [5]:
dcGan = DCGAN()

gen = dcGan.build_generator(dim=4, depth=1024)
gen.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               51712     
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 16384)             8404992   
_________________________________________________________________
activation_1 (Activation)    (None, 16384)             0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16384)             65536     
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 1024)        0

In [6]:
disc = dcGan.build_discriminator(width=64, height=64, depth=3)
disc.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       6272      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       524544    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 512)         2097664   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 1024)       

# Prepare data

In [7]:
# Nomalize preprocessor 
class NomalizePreprocessor:
    def __init__(self):
        pass

    def preprocess(self, image):
        return (image.astype("float") - 127.5) / 127.5

In [8]:
# Nomalize preprocessor 
class ResizePreprocessor:
    def __init__(self, new_size):
        self.new_size = new_size

    def preprocess(self, image):
        return cv2.resize(image, self.new_size)

# Training DC GAN

In [9]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist, mnist
from sklearn.utils import shuffle
from imutils import build_montages
import numpy as np
import argparse
import cv2
import os

In [10]:
OUTPUT_PATH = "output"
NUM_EPOCHS = 30
BATCH_SIZE = 128
INIT_LR = 2e-4

! mkdir "output"

mkdir: cannot create directory ‘output’: File exists


In [11]:
# Build generator model
gen = DCGAN.build_generator(dim=4, depth=1024)

# Build discriminator model
disc = DCGAN.build_discriminator(width=64, height=64, depth=3)

discOpt = Adam(learning_rate=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
disc.compile(loss="binary_crossentropy", optimizer=discOpt)

In [12]:
# Build GAN
disc.trainable = False  # FREEZE PARAMETERE
ganInput = Input(shape=(100,))
ganOutput = disc(gen(ganInput))
gan = Model(ganInput, ganOutput)

# compile the GAN
ganOpt = Adam(learning_rate=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
gan.compile(loss="binary_crossentropy", optimizer=discOpt)

In [13]:
normPro = NomalizePreprocessor()
resizePro = ResizePreprocessor(new_size=(64, 64))

In [ ]:
# Training process
benchmarkNoise = np.random.uniform(-1, 1, size=(64, 100))

# loop over the epochs
for epoch in range(0, NUM_EPOCHS):
    print("[INFO] starting epoch {} of {}...".format(epoch + 1, NUM_EPOCHS))
    batchesPerEpoch = int(images.shape[0] / BATCH_SIZE)

    for i in range(0, batchesPerEpoch):

        imageBatch = images[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        # preprocess real images
        process_imageBatch = []
        for image in imageBatch:
            image = resizePro.preprocess(image)
            image = normPro.preprocess(image)
            process_imageBatch.append(image)
        imageBatch = np.array(process_imageBatch)

        # Generator generate image
        noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
        genImages = gen.predict(noise, verbose=0)

        # Concat "real" images and "'fake" images
        X = np.concatenate((imageBatch, genImages))
        y = ([1] * BATCH_SIZE) + ([0] * BATCH_SIZE)
        y = np.reshape(y, (-1,))
        (X, y) = shuffle(X, y)

        # train the discriminator on the data
        discLoss = disc.train_on_batch(X, y)

        # Inverted label
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        fakeLabels = [1] * BATCH_SIZE
        fakeLabels = np.reshape(fakeLabels, (-1,))
        ganLoss = gan.train_on_batch(noise, fakeLabels)

        # End of Epoch
        if i == batchesPerEpoch - 1:
			
            p = [OUTPUT_PATH, "epoch_{}_output.png".format(str(epoch + 1).zfill(4))]

            print("[INFO] Step {}_{}: discriminator_loss={:.6f}, ""adversarial_loss={:.6f}".format(epoch + 1, i, discLoss, ganLoss))

            # make predictions on the benchmark noise, scale it back to [0, 255], and generate the montage
            plotImages = gen.predict(benchmarkNoise)
            plotImages = ((plotImages * 127.5) + 127.5).astype("uint8")
            vis = build_montages(plotImages, (64, 64), (8, 8))[0]

            # write the visualization to disk
            p = os.path.sep.join(p)
            print("path: ", p)
            cv2.imwrite(p, vis)